In [3]:
import numpy as np
import pandas as pd
from imutils import paths
import seaborn as sns
import matplotlib.pyplot as plt
import random

In [4]:
dataPaths = list(paths.list_files('.\ipl_csv2'))
dataPaths.pop(len(dataPaths) - 1)
dataPaths.pop(len(dataPaths) - 2)

def retrieve_past_data(team_a, team_b, num):
    req_df = []
    stadium_avg = {}
    unique_bowlers = set()
    unique_batsmen = set()

    for match_file in dataPaths:

        try:
            df = pd.read_csv(match_file, index_col='match_id')
            temp_a, temp_b = get_unique_players(df)
            unique_bowlers |= (temp_a) 
            unique_batsmen |= (temp_b) 
            df = preprocess(df)
            stadium_avg = get_stadium_avg(stadium_avg, df)
            if (df.iloc[1]['batting_team'] == team_a or df.iloc[1]['batting_team'] == team_b) and (df.iloc[1]['bowling_team'] == team_a or df.iloc[1]['bowling_team'] == team_b):
                if len(req_df) < num or num == -1:
                    req_df.append(df)
        
        except Exception as e:
            print(e)

    return req_df, stadium_avg, unique_batsmen, unique_bowlers


def preprocess(df):
    pp_df = df[df['ball'] <= 6.1]
    pp_df = pp_df[pp_df['innings'] <= 2]

    pp_df['total_runs'] = pp_df['runs_off_bat'] + pp_df['extras']

    return pp_df

#for stadium avg
def get_stadium_avg(stad_dict, df):
    venue = df.iloc[1]['venue']
    runs1 = df.loc[(df['ball'] < 6.1) & (df['innings'] == 1)]['total_runs'].sum()
    runs2 = df.loc[(df['ball'] < 6.1) & (df['innings'] == 2)]['total_runs'].sum()

    
    stad_dict.setdefault(venue,[]).append(runs1)
    stad_dict.setdefault(venue,[]).append(runs2)

    return stad_dict

def get_unique_players(df):
    bowlers = set(df['bowler'].unique().tolist())
    batsmen_striker = set(df['striker'].unique().tolist())
    batsmen_non_striker = set(df['non_striker'].unique().tolist())
    
    batsmen = batsmen_striker | (batsmen_non_striker)

    return bowlers, batsmen


In [5]:
#returns a list of dataframes between the two given teams
req_df, stad_avg, u_batsmen, u_bowler = retrieve_past_data("Chennai Super Kings", "Kings XI Punjab", -1)

C:\Users\rishi\AppData\Local\Programs\Python\Python39\lib\site-packages\IPython\core\interactiveshell.py:3357: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [6]:
#format the data for visualization (leave it out for now)

stad_avg['M.Chinnaswamy Stadium'].extend(stad_avg['M Chinnaswamy Stadium'])
# stad_avg.pop('M Chinnaswamy Stadium')

stad_avg['MA Chidambaram Stadium'].extend(stad_avg['MA Chidambaram Stadium, Chepauk'])
stad_avg['MA Chidambaram Stadium'].extend(stad_avg['MA Chidambaram Stadium, Chepauk, Chennai'])
# stad_avg.pop('MA Chidambaram Stadium, Chepauk')
# stad_avg.pop('MA Chidambaram Stadium, Chepauk, Chennai')

stad_avg['Punjab Cricket Association IS Bindra Stadium'].extend(stad_avg['Punjab Cricket Association IS Bindra Stadium, Mohali'])
stad_avg['Punjab Cricket Association IS Bindra Stadium'].extend(stad_avg['Punjab Cricket Association Stadium, Mohali'])
# stad_avg.pop('Punjab Cricket Association IS Bindra Stadium, Mohali')
# stad_avg.pop('Punjab Cricket Association Stadium, Mohali')

stad_avg['Rajiv Gandhi International Stadium'].extend(stad_avg['Rajiv Gandhi International Stadium, Uppal'])
# stad_avg.pop('Rajiv Gandhi International Stadium, Uppal')

stad_avg['Arun Jaitley Stadium'].extend(stad_avg['Feroz Shah Kotla'])
# stad_avg.pop('Feroz Shah Kotla')

stad_avg['Wankhede Stadium'].extend(stad_avg['Wankhede Stadium, Mumbai'])
# stad_avg.pop('Wankhede Stadium, Mumbai')

In [7]:
stad_rating = {}
for stad in stad_avg:
    stad_rating[stad] = float(np.mean(stad_avg[stad]))

stad_rating['M Chinnaswamy Stadium'] = stad_rating['M.Chinnaswamy Stadium']
stad_rating['MA Chidambaram Stadium, Chepauk'] = stad_rating['MA Chidambaram Stadium']
stad_rating['MA Chidambaram Stadium, Chepauk, Chennai'] = stad_rating['MA Chidambaram Stadium']
stad_rating['Punjab Cricket Association IS Bindra Stadium, Mohali'] = stad_rating['Punjab Cricket Association IS Bindra Stadium']
stad_rating['Punjab Cricket Association Stadium, Mohali'] = stad_rating['Punjab Cricket Association IS Bindra Stadium']
stad_rating['Rajiv Gandhi International Stadium, Uppal'] = stad_rating['Rajiv Gandhi International Stadium']
stad_rating['Feroz Shah Kotla'] = stad_rating['Arun Jaitley Stadium']
stad_rating['Wankhede Stadium, Mumbai'] = stad_rating['Wankhede Stadium']

print(stad_rating)

{'Rajiv Gandhi International Stadium, Uppal': 45.5859375, 'Maharashtra Cricket Association Stadium': 47.45238095238095, 'Saurashtra Cricket Association Stadium': 55.3, 'Holkar Cricket Stadium': 51.72222222222222, 'M.Chinnaswamy Stadium': 513.2125, 'Wankhede Stadium': 45.6554054054054, 'Eden Gardens': 46.26623376623377, 'M Chinnaswamy Stadium': 513.2125, 'Feroz Shah Kotla': 47.4527027027027, 'Punjab Cricket Association IS Bindra Stadium, Mohali': 48.42857142857143, 'Green Park': 55.0, 'Punjab Cricket Association IS Bindra Stadium': 48.42857142857143, 'Rajiv Gandhi International Stadium': 45.5859375, 'MA Chidambaram Stadium': 46.40677966101695, 'Sawai Mansingh Stadium': 45.04255319148936, 'Arun Jaitley Stadium': 47.4527027027027, 'Dr. Y.S. Rajasekhara Reddy ACA-VDCA Cricket Stadium': 40.96153846153846, 'Sheikh Zayed Stadium': 43.672413793103445, 'Dubai International Cricket Stadium': 44.71212121212121, 'Sharjah Cricket Stadium': 47.02777777777778, 'MA Chidambaram Stadium, Chepauk, Chenna

In [8]:
batsmen_record = {}
bowlers_record = {}

for match_file in dataPaths:
    df = pd.read_csv(match_file, index_col='match_id')
    df = df.loc[df['ball'] < 6.1]
    m_bowl, m_bat = get_unique_players(df) 

    for bat in m_bat:
        select = df.loc[df['striker'].str.contains(str(bat))]
        runs_scored = select['runs_off_bat'].sum() + select['extras'].sum()
        balls_faced = len(select.index)

        batsmen_record.setdefault(bat,[[],[]])[0].append(runs_scored)
        batsmen_record.setdefault(bat,[[],[]])[1].append(balls_faced)

    for bowl in m_bowl:
        select = df.loc[df['bowler'].str.contains(str(bowl))]

        total_runs_given = select['runs_off_bat'].sum() + select['extras'].sum()
        extras_given = select['extras'].sum()
        balls_bowled = len(select.index)
        wickets_taken = len(select.loc[select['wicket_type'].notnull()])

        bowlers_record.setdefault(bowl,[[],[],[],[]])[0].append(total_runs_given)
        bowlers_record.setdefault(bowl,[[],[],[],[]])[1].append(extras_given)
        bowlers_record.setdefault(bowl,[[],[],[],[]])[2].append(balls_bowled)
        bowlers_record.setdefault(bowl,[[],[],[],[]])[3].append(wickets_taken)


C:\Users\rishi\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\strings\accessor.py:101: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)
C:\Users\rishi\AppData\Local\Programs\Python\Python39\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
C:\Users\rishi\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\strings\accessor.py:101: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)
292


In [9]:
print(len(batsmen_record))
print(len(bowlers_record))


292
346


In [492]:

# Importing library
import csv
  
# data to be written row-wise in csv fil
data = list(stad_rating)

with open('stadium_rating.csv', 'w', newline='') as myfile:
     wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)

     for d in data:
        wr.writerow([d, stad_rating[d]])

In [522]:

# Importing library
import csv
  
# data to be written row-wise in csv fil
data = list(batsmen_record.keys())
data.sort()

with open('batsmen_record.csv', 'w', newline='') as myfile:
     wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)

     for d in data:
        wr.writerow([d, batsmen_record[d][0], batsmen_record[d][1]])

print(data)

['A Ashish Reddy', 'A Chopra', 'A Flintoff', 'A Mishra', 'A Mukund', 'A Symonds', 'AA Bilakhia', 'AA Jhunjhunwala', 'AB Agarkar', 'AB Barath', 'AB McDonald', 'AB de Villiers', 'AC Blizzard', 'AC Gilchrist', 'AC Voges', 'AD Hales', 'AD Mascarenhas', 'AD Mathews', 'AD Nath', 'AD Russell', 'AG Paunikar', 'AJ Finch', 'AL Menaria', 'AM Nayar', 'AM Rahane', 'AN Ghosh', 'AP Majumdar', 'AP Tare', 'AR Patel', 'AS Raut', 'AT Rayudu', 'AUK Pathan', 'Abdul Samad', 'Anirudh Singh', 'Ankit Sharma', 'Azhar Mahmood', 'B Chipli', 'BA Stokes', 'BB McCullum', 'BB Samantray', 'BJ Haddin', 'BJ Hodge', 'BJ Rohrer', 'BMAJ Mendis', 'BR Dunk', 'C Madan', 'C Munro', 'C de Grandhomme', 'CA Ingram', 'CA Lynn', 'CA Pujara', 'CH Gayle', 'CH Morris', 'CJ Anderson', 'CJ Ferguson', 'CL White', 'CM Gautam', 'D Padikkal', 'DA Miller', 'DA Warner', 'DB Das', 'DB Ravi Teja', 'DJ Bravo', 'DJ Harris', 'DJ Hooda', 'DJ Hussey', 'DJ Jacobs', 'DJ Thornely', 'DJG Sammy', 'DJM Short', 'DPMD Jayawardene', 'DR Martyn', 'DR Smith', 

In [523]:

# Importing library
import csv
  
# data to be written row-wise in csv fil
data = list(bowlers_record.keys())
data.sort()

with open('bowlers_record.csv', 'w', newline='') as myfile:
     wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)

     for d in data:
        wr.writerow([d, bowlers_record[d][0], bowlers_record[d][1], bowlers_record[d][2], bowlers_record[d][3]])

print(data)

['A Ashish Reddy', 'A Chandila', 'A Choudhary', 'A Dananjaya', 'A Flintoff', 'A Kumble', 'A Mishra', 'A Mithun', 'A Nehra', 'A Nel', 'A Nortje', 'A Singh', 'A Symonds', 'A Uniyal', 'A Zampa', 'AA Chavan', 'AA Jhunjhunwala', 'AA Noffke', 'AB Agarkar', 'AB Dinda', 'AB McDonald', 'AC Thomas', 'AC Voges', 'AD Mascarenhas', 'AD Mathews', 'AD Russell', 'AF Milne', 'AG Murtaza', 'AJ Finch', 'AJ Tye', 'AL Menaria', 'AM Nayar', 'AM Salvi', 'AN Ahmed', 'AP Dole', 'AR Patel', 'AS Joseph', 'AS Rajpoot', 'Anand Rajan', 'Ankit Sharma', 'Ankit Soni', 'Anureet Singh', 'Arshdeep Singh', 'Avesh Khan', 'Azhar Mahmood', 'B Akhil', 'B Geeves', 'B Kumar', 'B Laughlin', 'B Lee', 'B Stanlake', 'BA Bhatt', 'BA Stokes', 'BAW Mendis', 'BB Sran', 'BCJ Cutting', 'BE Hendricks', 'BJ Hodge', 'BW Hilfenhaus', 'Basil Thampi', 'Bipul Sharma', 'C de Grandhomme', 'CH Gayle', 'CH Morris', 'CJ Anderson', 'CJ Dala', 'CJ Green', 'CJ Jordan', 'CJ McKay', 'CK Langeveldt', 'CR Brathwaite', 'CR Woakes', 'CRD Fernando', 'CV Varun

In [318]:
# stad_names = list(stad_avg.keys())
# keys = pd.Series(stad_names, dtype="category")
# vals = [float(len(stad_avg[k])) for k in keys.cat.categories]


# plt.figure(figsize=(20, 10))
# sns.barplot(x=keys.cat.codes, y=vals)

In [319]:
# dict(enumerate(keys.cat.categories))

In [527]:
def get_venue_score(df):
    venue = df.iloc[1]['venue']
    #assign venue scores here

    return stad_rating[venue]

def get_batsmen_scores(df):
    u_bat = list(set(df['striker'].unique()) | set(df['non_striker'].unique()))

    rating = []

    for batsman in u_bat:
        rating.append(np.mean(batsmen_record[batsman][0]))
    
    return rating
    
def get_bowlers_scores(df):

    u_bowlers = list(df['bowler'].unique())

    rating = []

    for (i,bowler) in enumerate(u_bowlers, 0):
        rating.append(random.uniform(0, 1))
    
    return rating

def get_wickets(df):
    return len(get_batsmen_scores(df)) - 2

In [580]:
#For cross validation, use the last of the req_df as test data and use the rest for training
test = req_df[len(req_df) - 1]
train_set = req_df

#test_x is the test input similar to actual input
test_batsmen = list(set(test.loc[(test['ball'] < 6.1) & (test['innings'] == 1)]['striker'].unique()) | set(test.loc[(test['ball'] < 6.1) & (test['innings'] == 1)]['non_striker'].unique()))
test_bowlers = test.loc[(test['ball'] < 6.1) & (test['innings'] == 1)]['bowler'].unique()
test_x = pd.DataFrame({'venue': [test.iloc[2]['venue']], 'innings': [1], 'batting_team': [test.iloc[2]['batting_team']], 'bowling_team':[test.iloc[2]['bowling_team']], 'batsmen': [test_batsmen], 'bowlers': [test_bowlers]})
test_y = [df.loc[(df['ball'] < 6.1) & (df['innings'] == 1)]['total_runs'].sum()]

#used to find the innings of the training match we are going to use for training
team_a = test.iloc[1]['batting_team']
team_b = test.iloc[1]['bowling_team']

train_x = []
train_y = []

for df in train_set:
    match_id = df.index[0]

    #innings for which we are going to use for training
    inn = df[(df['batting_team'] == team_a) & (df['bowling_team'] == team_b)]['innings'].iloc[0] 

    runs_x = df.loc[(df['ball'] < 6.1) & (df['innings'] == inn)]['total_runs'].sum() #runs of training innings
    runs_target = df.loc[(df['ball'] < 6.1) & (df['innings'] == 3 - inn)]['total_runs'].sum() #runs of other innings
    
    #MODEL CODE GOES HERE


    #convert the df into usable training data with required features
    df_train_inn = df.loc[(df['ball'] < 6.1) & (df['innings'] == inn)]

    # to add bowler rating: 'bowlers_rating': [numpy.array(get_bowlers_scores(df_train_inn))] 
    temp = pd.DataFrame({'venue': [get_venue_score(df)], 'batsmen_rating': [np.sum(numpy.array(get_batsmen_scores(df_train_inn)))], 'wickets': [get_wickets(df_train_inn)], 'other_runs': [runs_target], 'chasing': [0 if inn == 1 else 1]})

    train_x.append(temp.values.tolist()[0])
    train_y.append(runs_x)

print(train_x)
print(train_y)

[[48.098214285714285, 38.129667259334525, 0.0, 53.0, 0.0], [47.285714285714285, 64.74773736127558, 3.0, 33.0, 0.0], [46.21186440677966, 60.7054248350921, 2.0, 54.0, 1.0], [48.098214285714285, 38.129667259334525, 0.0, 42.0, 1.0], [43.310344827586206, 51.7054248350921, 1.0, 57.0, 0.0], [44.45454545454545, 32.398338220918866, 0.0, 60.0, 0.0], [48.098214285714285, 25.64835164835165, 0.0, 53.0, 1.0], [46.21186440677966, 54.41758241758242, 2.0, 51.0, 1.0], [45.29054054054054, 67.54042524702189, 4.0, 35.0, 0.0], [45.0, 47.97234791352439, 1.0, 57.0, 1.0], [45.86666666666667, 48.52042483660131, 1.0, 50.0, 1.0], [46.21186440677966, 35.201530612244895, 1.0, 42.0, 0.0], [40.55555555555556, 37.255761381475665, 1.0, 49.0, 0.0], [48.098214285714285, 45.77194816053512, 1.0, 31.0, 1.0], [46.21186440677966, 25.30923076923077, 0.0, 48.0, 0.0], [40.55555555555556, 30.9775, 0.0, 25.0, 1.0], [48.098214285714285, 53.79050813008129, 2.0, 38.0, 0.0], [46.21186440677966, 38.934230769230766, 1.0, 36.0, 1.0], [43

In [576]:
from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error
import xgboost as xgb
import lightgbm as lgb

In [577]:
lasso = make_pipeline(RobustScaler(), Lasso(alpha =0.0005, random_state=1))
ENet = make_pipeline(RobustScaler(), ElasticNet(alpha=0.0005, l1_ratio=.9, random_state=3))
KRR = KernelRidge(alpha=0.6, kernel='polynomial', degree=2, coef0=2.5)
GBoost = GradientBoostingRegressor(n_estimators=3000, learning_rate=0.05,
                                   max_depth=4, max_features='sqrt',
                                   min_samples_leaf=15, min_samples_split=10, 
                                   loss='huber', random_state =5)
model_xgb = xgb.XGBRegressor(colsample_bytree=0.4603, gamma=0.0468, 
                             learning_rate=0.05, max_depth=3, 
                             min_child_weight=1.7817, n_estimators=2200,
                             reg_alpha=0.4640, reg_lambda=0.8571,
                             subsample=0.5213, silent=1,
                             random_state =7, nthread = -1)
model_lgb = lgb.LGBMRegressor(objective='regression',num_leaves=5,
                              learning_rate=0.05, n_estimators=720,
                              max_bin = 55, bagging_fraction = 0.8,
                              bagging_freq = 5, feature_fraction = 0.2319,
                              feature_fraction_seed=9, bagging_seed=9,
                              min_data_in_leaf =6, min_sum_hessian_in_leaf = 11)

In [581]:
n_folds = 5

def rmsle_cv(model):
    kf = KFold(n_folds, shuffle=True, random_state=42).get_n_splits(train_x)
    rmse= np.sqrt(-cross_val_score(model, train_x, train_y, scoring="neg_mean_squared_error", cv = kf))
    return(rmse)

In [582]:
score = rmsle_cv(lasso)
print("\nLasso score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))


Lasso score: 13.3384 (5.5679)



In [594]:
from sklearn.linear_model import Lasso

#Initializing the Lasso Regressor with Normalization Factor as True
lasso_reg = Lasso(normalize=True)

#Fitting the Training data to the Lasso regressor

lasso_reg.fit(train_x[1:], train_y[1:])

#Predicting for X_test
y_pred_lass =lasso_reg.predict([train_x[6]])

#Printing the Score with RMLSE
print(y_pred_lass, train_y[6])

[46.4733186] 56


NameError: name 'L_layer_model' is not defined